In [1]:
from openslide import OpenSlide
from PIL import Image
import numpy as np

import os
from time import time
# from joblib import Parallel, delayed
# import cv2

In [2]:
svs_src = r'\\motherserverdw\Kyu_Sync\Research\Active\Aging_organized\svs\svs_back'
svss = [os.path.join(svs_src,_) for _ in os.listdir(svs_src) if _.endswith(('ndpi','svs')) ]
# svss = [os.path.join(svs_src,_) for _ in os.listdir(svs_src) if _.endswith('ndpi') ]
svs_dst = r'\\motherserverdw\Kyu_Sync\Research\Active\Aging_organized\svs\svs_back_2um'

In [3]:
def svs2tiff(svs):
    src,fn = os.path.split(svs)
    fn,ext = os.path.splitext(fn)
    fn1 = fn + '.tif'
    # fn2 = fn + '.jp2'
    if os.path.exists(os.path.join(svs_dst,fn1)): return
    print(fn)
    svs_obj = OpenSlide(svs)

    svs_img = svs_obj.read_region(location=(0,0),level=0,size=svs_obj.level_dimensions[0]).convert('RGB')

    resize_factorx = 2/float(svs_obj.properties['openslide.mpp-x']) #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = 2/float(svs_obj.properties['openslide.mpp-y'])
    resize_dimension = tuple([int(np.ceil(svs_obj.dimensions[0]/resize_factorx)),int(np.ceil(svs_obj.dimensions[1]/resize_factory))])

    svs_img = svs_img.resize(resize_dimension,resample=Image.NEAREST)
    svs_img.save(os.path.join(svs_dst,fn1),resolution=1,resolution_unit=1,quality=100,compression=None)
    # svs_img2 = np.array(svs_img)
    # retval = cv2.imwrite(os.path.join(svs_dst,fn2),svs_img2,[cv2.IMWRITE_JPEG2000_COMPRESSION_X1000,800])


In [4]:
svss2=svss[40:80]

In [ ]:
st = time()
# Parallel(n_jobs=5, prefer="threads")(delayed(svs2tiff)(svs) for svs in svss)
for svs in svss2:
    try:
        svs2tiff(svs)
    except:
        print(svs)
        continue


wm34_b5_01


In [ ]:
print("{:.2f} sec elapsed for {:d} images at 10x".format(time()-st,len(svss)))